## <font color='red'>USING SPARK </font>

## This notebook contains the flow for traditional machine learning steps  scale to spark:

 1. data import
 2. data wrangling for NLP
 3. train-test-hold data split
 4. Imbalanced data treatment 
 5. BOW (bag of words) machine learning 
     NaviebBayes 
     hyper-parameter turning  on GBX and LogisticRegression
 6. Simple Network 
     

In [5]:
import pyspark

print(type(spark),type(sc),type(sqlCtx))

<class 'pyspark.sql.session.SparkSession'> <class 'pyspark.context.SparkContext'> <class 'pyspark.sql.context.SQLContext'>


In [6]:
spark
sc
sqlCtx
pyspark.__version__

'2.4.0'

### STEP 1 : DATA IMPORT

In [25]:
import pandas as pd

train_data = pd.read_csv('data/train.csv')
print('Training data set: ' + str(len(train_data)))


Training data set: 1306122


In [21]:
train_data.head(3)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0


In [22]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 3 columns):
qid              1306122 non-null object
question_text    1306122 non-null object
target           1306122 non-null int64
dtypes: int64(1), object(2)
memory usage: 29.9+ MB


In [23]:
# import pyspark
# from pyspark.sql.types import StructType, StructField
# from pyspark.sql.types import DoubleType, IntegerType, StringType

# schema = StructType([
#     StructField("qid", StringType()),
#     StructField("question_text", StringType()),
#     StructField("target", IntegerType())
# ])

# spark_train_1 = spark.read.csv('data/train.csv', header = True, inferSchema = True)
# spark_train_2 =  spark.read.load("data/train.csv", parserLib= "univocity", charset="ISO-8859-1", format = "csv", sep = ",", schema = schema, header="true")

# print('Training data set: ' + str(spark_train_data1.count()))
# print('Training data set: ' + str(spark_train_data2.count()))


<B>The above is not working, as for now, the spark file loading has problem on double quote, the work around is to load from pandas dataframe [CSV I/O: does not respect RFC 4180](https://issues.apache.org/jira/browse/SPARK-22236)</B>

In [3]:
spark_train_data = sqlCtx.createDataFrame(train_data)

In [4]:
spark_train_data.printSchema()
spark_train_data.describe().show()
print('Training data set: ' + str(spark_train_data.count()))
spark_train_data.head(3)

root
 |-- qid: string (nullable = true)
 |-- question_text: string (nullable = true)
 |-- target: long (nullable = true)

+-------+--------------------+--------------------+-------------------+
|summary|                 qid|       question_text|             target|
+-------+--------------------+--------------------+-------------------+
|  count|             1306122|             1306122|            1306122|
|   mean|            Infinity|                null|0.06187017751787352|
| stddev|                 NaN|                null| 0.2409197025783378|
|    min|00002165364db923c7e6|I want to blow t...|                  0|
|    max|ffffed09fedb5088744a|￼￼Assuming that a...|                  1|
+-------+--------------------+--------------------+-------------------+

Training data set: 1306122


[Row(qid='00002165364db923c7e6', question_text='How did Quebec nationalists see their province as a nation in the 1960s?', target=0),
 Row(qid='000032939017120e6e44', question_text='Do you have an adopted dog, how would you encourage people to adopt and not shop?', target=0),
 Row(qid='0000412ca6e4628ce2cf', question_text='Why does velocity affect time? Does velocity affect space geometry?', target=0)]

In [5]:
#preserve the data for later
spark_train_data.write.mode('overwrite').parquet("data/spark_train_data.parquet")


### STEP 2.1 DATA WRANGLING, simple

In [12]:
%%time

spark_train_data = spark.read.parquet("data/spark_train_data.parquet")
spark_train_data.printSchema()
spark_train_data.describe().show()
print('Training data set: ' + str(spark_train_data.count()))

root
 |-- qid: string (nullable = true)
 |-- question_text: string (nullable = true)
 |-- target: long (nullable = true)

+-------+--------------------+--------------------+-------------------+
|summary|                 qid|       question_text|             target|
+-------+--------------------+--------------------+-------------------+
|  count|             1306122|             1306122|            1306122|
|   mean|            Infinity|                null|0.06187017751787352|
| stddev|                 NaN|                null| 0.2409197025783378|
|    min|00002165364db923c7e6|I want to blow t...|                  0|
|    max|ffffed09fedb5088744a|￼￼Assuming that a...|                  1|
+-------+--------------------+--------------------+-------------------+

Training data set: 1306122
CPU times: user 11.8 ms, sys: 1.12 ms, total: 12.9 ms
Wall time: 13.6 s


In [4]:
! pip install contractions

pyspark 2.4.0 requires py4j==0.10.7, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import lower, col
import contractions
import re

def remove_non_ascii(text):
    return ''.join(i for i in text if ord(i)<128)
def remove_white_space(text):
    return text.strip().strip('\t\n')
def remove_special_character(text):
    return re.sub('[^A-Za-z0-9\s]+', '', text)
# F1 perform contractions fix and three functions as above, to lower case will use the sql functon from pyspark
dataClean = udf(lambda x: remove_special_character(contractions.fix(remove_white_space(remove_non_ascii(x)))), StringType())


In [14]:
%%time

from pyspark.sql.types import IntegerType

spark_train_data = spark_train_data.withColumn('origin', col('question_text'))
spark_train_data = spark_train_data.withColumn('label', col('target').cast('int'))
spark_train_data = spark_train_data.withColumn("question_text",lower(dataClean(spark_train_data["question_text"])))


CPU times: user 4.93 ms, sys: 7.26 ms, total: 12.2 ms
Wall time: 88.7 ms


In [ ]:
%%time

#spark_train_data.show(2)

### Step 2.2 spark tokenization, stop word removing and Naivebayes

In [15]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

#tokenizer = Tokenizer(inputCol="question_text", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="question_text", outputCol="words", pattern="\\W")
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")
# reduce calculation time and sparse, set vocab size to 65536
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=65536) 


In [9]:
%%time

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])
pipelineFit = pipeline.fit(spark_train_data)
spark_train_data = pipelineFit.transform(spark_train_data)


CPU times: user 113 ms, sys: 28.6 ms, total: 141 ms
Wall time: 4min 20s


In [20]:
%%time

countTokens = udf(lambda words: len(words), IntegerType())
spark_train_data = spark_train_data.withColumn("tokens", countTokens(col("filtered")))


CPU times: user 8.22 ms, sys: 0 ns, total: 8.22 ms
Wall time: 59.2 ms


In [10]:
%%time

spark_train_data.show(2)

+--------------------+--------------------+------+--------------------+-----+--------------------+--------------------+--------------------+
|                 qid|       question_text|target|              origin|label|               words|            filtered|            features|
+--------------------+--------------------+------+--------------------+-----+--------------------+--------------------+--------------------+
|40073653a1341fd91860|name some small t...|     0|Name some small t...|    0|[name, some, smal...|[name, small, tow...|(65536,[3,64,93,2...|
|4007527f139de5f2091e|a 15minute boat r...|     0|A 15-minute boat ...|    0|[a, 15minute, boa...|inute, boat, ...|(65536,[284,450,4...|
+--------------------+--------------------+------+--------------------+-----+--------------------+--------------------+--------------------+
only showing top 2 rows

CPU times: user 1.92 ms, sys: 619 µs, total: 2.54 ms
Wall time: 2.36 s


In [11]:
%%time

#preserve the data for later
spark_train_data = spark_train_data.select("features","label")
spark_train_data.write.mode('overwrite').parquet("data/spark_train_data_count.parquet")


CPU times: user 21.1 ms, sys: 6.79 ms, total: 27.8 ms
Wall time: 4min 18s


In [16]:
from pyspark.ml.feature import HashingTF, IDF

# reduce calculation time and sparse, restrict the features
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=65536)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)


In [17]:
%%time

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf])
pipelineFit = pipeline.fit(spark_train_data)
spark_train_data = pipelineFit.transform(spark_train_data)


CPU times: user 118 ms, sys: 16.5 ms, total: 134 ms
Wall time: 4min 9s


In [ ]:
%%time

#spark_train_data.show(2,False) 

In [18]:
%%time

#preserve the data for later
spark_train_data = spark_train_data.select("features","label")
spark_train_data.write.mode('overwrite').parquet("data/spark_train_data_tfidf.parquet")


CPU times: user 22.2 ms, sys: 5.15 ms, total: 27.4 ms
Wall time: 4min 14s


In [ ]:
%%time

#spark_train_data.show(2,False) 

### STEP 3 TRAIN-TEST-HOLD DATA SPLIT

In [1]:
%%time

spark_train_data = spark.read.parquet("data/spark_train_data_count.parquet")
spark_train_data.printSchema()
spark_train_data.describe().show()
print('Training data set: ' + str(spark_train_data.count()))
spark_train_data.show(2,False) 

root
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|            1306122|
|   mean|0.06187017751787352|
| stddev|0.24091970257833784|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

Training data set: 1306122
+--------------------------------------------------------+-----+
|features                                                |label|
+--------------------------------------------------------+-----+
|(65536,[22,1309,35260],[1.0,1.0,1.0])                   |0    |
|(65536,[441,1046,1570,4267,28286],[1.0,1.0,1.0,1.0,1.0])|0    |
+--------------------------------------------------------+-----+
only showing top 2 rows

CPU times: user 13 ms, sys: 199 µs, total: 13.2 ms
Wall time: 13.6 s


In [2]:
%%time

insincere = spark_train_data.filter(spark_train_data['label'] > 0)
sincere = spark_train_data.filter(spark_train_data['label'] <= 0)


CPU times: user 2.57 ms, sys: 0 ns, total: 2.57 ms
Wall time: 862 ms


In [21]:
print(insincere.count(), sincere.count())

80810 1225312


In [3]:
%%time

(train_1, hold_1) = insincere.randomSplit([0.8, 0.2], seed = 42)
(train_0, hold_0) = sincere.randomSplit([0.8, 0.2], seed = 42)
hold = hold_1.unionAll(hold_0)

(train_1, test_1) = train_1.randomSplit([0.8, 0.2], seed = 42)
(train_0, test_0) = train_0.randomSplit([0.8, 0.2], seed = 42)
test = test_1.unionAll(test_0)

(train_1_25, train_1_75) = train_1.randomSplit([0.25, 0.75], seed = 42)
(train_0_25, train_0_75) = train_0.randomSplit([0.25, 0.75], seed = 42)
train_25 = train_1_25.unionAll(train_0_25)
train = train_1.unionAll(train_0)

print(hold.count(), test.count(), train_25.count(), train.count())

260061 208417 209080 837644
CPU times: user 20.6 ms, sys: 309 µs, total: 20.9 ms
Wall time: 41.6 s


## STEP 4 IMBALANCED DATA TREATMENT

<B>simple down sampling</B> 

In [24]:
%%time

size =  2 * (train_1.count() / (train_1.count() + train_0.count()))
(train_0_d, _) = train_0.randomSplit([size, 1-size], seed = 42)
train_balanced = train_1.unionAll(train_0_d)
print(train_balanced.count())

149182
CPU times: user 6.18 ms, sys: 0 ns, total: 6.18 ms
Wall time: 25.7 s


In [4]:
(train_1_10, _) = train_1.randomSplit([0.10, 0.90], seed = 42)
size =   train_1_10.count() / train_0.count()
(train_0_10, _) = train_0.randomSplit([size, 1-size], seed = 42)
train_small = train_1_10.unionAll(train_0_10)
print(train_small.count())

10370


In [5]:
%%time

print(train_small.groupBy('label').count().collect())
train_small.write.mode('overwrite').parquet("data/train_small.parquet")


[Row(label=1, count=5170), Row(label=0, count=5200)]
CPU times: user 14.1 ms, sys: 4.81 ms, total: 18.9 ms
Wall time: 27.5 s


In [35]:
%%time

print(hold.groupBy('label').count().collect())
hold.write.mode('overwrite').parquet("data/hold.parquet")

[Row(label=1, count=16056), Row(label=0, count=244005)]
CPU times: user 10.1 ms, sys: 1 ms, total: 11.1 ms
Wall time: 21.3 s


In [36]:
%%time

print(train_balanced.groupBy('label').count().collect())
train_balanced.write.mode('overwrite').parquet("data/train_balanced.parquet")

[Row(label=1, count=51833), Row(label=0, count=97349)]
CPU times: user 8.71 ms, sys: 1.9 ms, total: 10.6 ms
Wall time: 22.4 s


In [37]:
%%time

print(train_25.groupBy('label').count().collect())
train_25.write.mode('overwrite').parquet("data/train_25.parquet")

[Row(label=1, count=12799), Row(label=0, count=196281)]
CPU times: user 4.32 ms, sys: 5.82 ms, total: 10.1 ms
Wall time: 21.4 s


In [38]:
%%time

print(test.groupBy('label').count().collect())
test.write.mode('overwrite').parquet("data/test.parquet")

[Row(label=1, count=12921), Row(label=0, count=195496)]
CPU times: user 7.41 ms, sys: 5.4 ms, total: 12.8 ms
Wall time: 22.5 s


In [39]:
%%time

print(train.groupBy('label').count().collect())
train.write.mode('overwrite').parquet("data/train.parquet")

[Row(label=1, count=51833), Row(label=0, count=785811)]
CPU times: user 15 ms, sys: 1.77 ms, total: 16.8 ms
Wall time: 25.1 s


### STEP 5.1 BAG OF WORDS  MACHINE LEARNING  NaviebBayes, LogisticRegression and SVC

In [1]:
%%time

hold = spark.read.parquet("data/hold.parquet")
test = spark.read.parquet("data/test.parquet")
train_balanced = spark.read.parquet("data/train_balanced.parquet")
train_25 = spark.read.parquet("data/train_25.parquet")


CPU times: user 8.12 ms, sys: 548 µs, total: 8.67 ms
Wall time: 9.51 s


In [2]:
train_small = spark.read.parquet("data/train_small.parquet")

<B>clean up dataset as there are zero features from feature transform pipeline</B>

In [3]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf

def is_null(text):
    return text[1] < 1

featureClean = udf(lambda x: not is_null(x), BooleanType())


In [4]:
%%time

hold = hold.filter(featureClean(hold.features))
test = test.filter(featureClean(test.features))
train_25 = train_25.filter(featureClean(train_25.features))
train_balanced = train_balanced.filter(featureClean(train_balanced.features))


CPU times: user 229 ms, sys: 27.2 ms, total: 256 ms
Wall time: 599 ms


In [5]:
train_small = train_small.filter(featureClean(train_small.features))

In [81]:
%%time

print(hold.groupBy('label').count().collect())
print(test.groupBy('label').count().collect())
print(train_25.groupBy('label').count().collect())
print(train_balanced.groupBy('label').count().collect())


[Row(label=1, count=122), Row(label=0, count=12236)]
[Row(label=1, count=113), Row(label=0, count=9785)]
[Row(label=1, count=109), Row(label=0, count=9718)]
[Row(label=1, count=411), Row(label=0, count=4853)]
CPU times: user 79.9 ms, sys: 20.7 ms, total: 101 ms
Wall time: 57.7 s


In [9]:
print(train_small.groupBy('label').count().collect())


[Row(label=1, count=48), Row(label=0, count=263)]


<B>NaiveBayes</B>

In [132]:
%%time

from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramMaps = ParamGridBuilder()\
    .addGrid(NaiveBayes.modelType, ['multinomial', 'bernoulli'])\
    .addGrid(NaiveBayes.smoothing, [0.1, 0.5, 1.0, 1.5])\
    .build()
evaluator = BinaryClassificationEvaluator(metricName = "areaUnderROC")

crossval = CrossValidator(estimator=NaiveBayes(),estimatorParamMaps= paramMaps, evaluator= evaluator, numFolds=5)
cvModel = crossval.fit(train_balanced)
print(cvModel.avgMetrics)


[0.33436096127408216, 0.33436096127408216, 0.3343609612740822, 0.3343609612740821, 0.3343609612740822, 0.33436096127408216, 0.3343609612740822, 0.33436096127408216]
CPU times: user 1.66 s, sys: 314 ms, total: 1.98 s
Wall time: 2min 41s


In [133]:
print(crossval.explainParams())

collectSubModels: Param for whether to collect a list of sub-models trained during tuning. If set to false, then only the single best sub-model will be available after fitting. If set to true, then all sub-models will be available. Warning: For large models, collecting all sub-models can cause OOMs on the Spark driver. (default: False)
estimator: estimator to be cross-validated (current: NaiveBayes_eed55a48a064)
estimatorParamMaps: estimator param maps (current: [{Param(parent='undefined', name='modelType', doc='The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli.'): 'multinomial', Param(parent='undefined', name='smoothing', doc='The smoothing parameter, should be >= 0, default is 1.0'): 0.1}, {Param(parent='undefined', name='modelType', doc='The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli.'): 'multinomial', Param(parent='undefined', name='smoothing', doc='The smoothing param

In [134]:
%%time

bestNaive = cvModel.bestModel
train_pred = bestNaive.transform(train_balanced)
train_auc = evaluator.evaluate(train_pred)
print(train_auc)
test_pred = bestNaive.transform(test)
test_auc = evaluator.evaluate(test_pred)
print(test_auc)
print("  ")
print(test_pred.count())
print(test_pred.filter(test_pred.label - test_pred.prediction == 0).count())
print(test_pred.filter(test_pred.label - test_pred.prediction != 0.0).count())

0.3551042999965405
0.3685485730823327
  
9898
9787
111
CPU times: user 165 ms, sys: 18 ms, total: 183 ms
Wall time: 55.7 s


<B>LogisticRegression</B>

In [135]:
%%time

from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

estimator = LogisticRegression()
paramMaps = ParamGridBuilder()\
            .addGrid(LogisticRegression.maxIter, [10, 50, 100])\
            .addGrid(LogisticRegression.regParam, [0.01, 0.1, 1.0, 10])\
            .addGrid(LogisticRegression.elasticNetParam, [0, 0.3, 0.6, 1])\
            .build()
evaluator = BinaryClassificationEvaluator(metricName = "areaUnderROC")
#evaluator = RegressionEvaluator(metricName = "r2")  f1|weightedPrecision|weightedRecall|accuracy)
# evaluator = MulticlassClassificationEvaluator(metricName = "f1")


crossval = CrossValidator(estimator = estimator,estimatorParamMaps = paramMaps, evaluator = evaluator, numFolds = 5)
cvModel = crossval.fit(train_balanced)
print(cvModel.avgMetrics)


[0.8177076381522643, 0.8177076381522642, 0.8177076381522643, 0.8177076381522641, 0.8177076381522641, 0.8177076381522642, 0.8177076381522642, 0.8177076381522641, 0.8177076381522641, 0.8177076381522641, 0.8177076381522644, 0.817707638152264, 0.8177076381522643, 0.8177076381522641, 0.8177076381522643, 0.8177076381522641, 0.8177076381522644, 0.8177076381522643, 0.8177076381522645, 0.8177076381522641, 0.8177076381522641, 0.8177076381522641, 0.817707638152264, 0.8177076381522641, 0.817707638152264, 0.8177076381522641, 0.817707638152264, 0.8177076381522641, 0.8177076381522645, 0.8177076381522642, 0.8177076381522642, 0.8177076381522641, 0.8177076381522643, 0.8177076381522641, 0.8177076381522643, 0.8177076381522642, 0.8177076381522643, 0.8177076381522641, 0.8177076381522643, 0.8177076381522641, 0.8177076381522641, 0.8177076381522641, 0.8177076381522642, 0.8177076381522641, 0.8177076381522639, 0.8177076381522642, 0.8177076381522645, 0.8177076381522641]
CPU times: user 16.7 s, sys: 3.83 s, total:

In [136]:
print(crossval.explainParams())

collectSubModels: Param for whether to collect a list of sub-models trained during tuning. If set to false, then only the single best sub-model will be available after fitting. If set to true, then all sub-models will be available. Warning: For large models, collecting all sub-models can cause OOMs on the Spark driver. (default: False)
estimator: estimator to be cross-validated (current: LogisticRegression_d1aa6f6acd9e)
estimatorParamMaps: estimator param maps (current: [{Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='undefined', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, {Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='undefined', name='regParam', doc='regularization 

In [137]:
%%time

bestLogistic = cvModel.bestModel
train_pred = bestLogistic.transform(train_balanced)
train_auc = evaluator.evaluate(train_pred)
print(train_auc)
test_pred = bestLogistic.transform(test)
test_auc = evaluator.evaluate(test_pred)
print(test_auc)
print("  ")
print(test_pred.count())
print(test_pred.filter(test_pred.label - test_pred.prediction == 0).count())
print(test_pred.filter(test_pred.label - test_pred.prediction != 0.0).count())

# calculate results 
# r = train_pred.stat.corr("prediction", "label")
# print("R-sqaured: " + str(r**2))


0.9999999999999999
0.8124092773388885
  
9898
9447
451
CPU times: user 178 ms, sys: 31.5 ms, total: 209 ms
Wall time: 55 s


In [138]:
%%time

from pyspark.ml.classification import LogisticRegressionSummary
# Extract the summary from the returned LogisticRegressionModel instance trained

trainingSummary = bestLogistic.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))


objectiveHistory:
0.27404868371536206
0.1273490866044008
0.08254727768078535
0.06468886600160602
0.04152607490129559
0.034992609702753315
0.030756827356619738
0.018648170286254928
0.010391922762000032
0.0067322612803490545
0.004209617677492705
0.0027907206806629913
0.0018466033438254569
0.0009730560886763704
0.0006127332257943605
0.00039503086426880793
0.00026347696929724464
0.0001622487233018019
3.579882784406424e-05
2.5525197994162546e-05
1.0995128455427055e-05
5.820016707578058e-06
2.890317775447016e-06
1.6821906947139898e-06
9.391355946984973e-07
4.708336356085494e-07
2.8172511220718963e-07
9.791689423050575e-08
6.46949119449137e-08
2.952431128776676e-08
1.4439837320307307e-08
4.608749084991078e-09
3.099071923794927e-09
+--------------------+------------------+
|                 FPR|               TPR|
+--------------------+------------------+
|                 0.0|               0.0|
|                 0.0|0.8077858880778589|
|                 0.0|0.9245742092457421|
|0.00350298784

<B>LinearSVC</B>

In [ ]:
%%time

from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

estimator = LinearSVC()
paramMaps = ParamGridBuilder()\
            .addGrid(LinearSVC.maxIter, [10, 50, 100])\
            .addGrid(LinearSVC.regParam, [0.01, 0.1, 1.0, 10])\
            .addGrid(LinearSVC.aggregationDepth, [10, 20, 30])\
            .build()
evaluator = BinaryClassificationEvaluator(metricName = "areaUnderROC")
# evaluator = MulticlassClassificationEvaluator(metricName = "f1")

crossval = CrossValidator(estimator = estimator,estimatorParamMaps = paramMaps, evaluator = evaluator, numFolds = 5)
cvModel = crossval.fit(train_balanced)
print(cvModel.avgMetrics)


[0.8268174687231141, 0.8261320074771027, 0.8286958407057525, 0.82616277354784, 0.8235896624409202, 0.8260151277894385, 0.8297603759650024, 0.8304491657812564, 0.8248175814843852, 0.8293411329581271, 0.8270774989909728, 0.8248659809571827, 0.8278779451161176, 0.8283488329454913, 0.8242101079768207, 0.8268451585675043, 0.8260037834751355, 0.8282770782943996, 0.826462639315843, 0.8264841931022189, 0.8271715238527055, 0.826980489751999, 0.8276612721927648, 0.8274341102260567, 0.8243700402517584, 0.8267886340341721, 0.8263819259054923, 0.8259909458069702, 0.823533730807501, 0.824767651906768, 0.8291546755769761, 0.8277425460521606, 0.8267588653281482, 0.8240879304452569, 0.8270839730134444, 0.8258897348204209]
CPU times: user 41.3 s, sys: 11.4 s, total: 52.6 s
Wall time: 1h 48min 58s


In [ ]:
print(crossval.explainParams())

collectSubModels: Param for whether to collect a list of sub-models trained during tuning. If set to false, then only the single best sub-model will be available after fitting. If set to true, then all sub-models will be available. Warning: For large models, collecting all sub-models can cause OOMs on the Spark driver. (default: False)
estimator: estimator to be cross-validated (current: LinearSVC_ae3b83dcdd4f)
estimatorParamMaps: estimator param maps (current: [{Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='undefined', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 10}, {Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='undefined', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='undefined', name='aggregationDepth', doc='suggested depth

In [ ]:
%%time

bestSVC = cvModel.bestModel
train_pred = bestSVC.transform(train_balanced)
train_auc = evaluator.evaluate(train_pred)
print(train_auc)
test_pred = bestSVC.transform(test)
test_auc = evaluator.evaluate(test_pred)
print(test_auc)
print("  ")
print(test_pred.count())
print(test_pred.filter(test_pred.label - test_pred.prediction == 0).count())
print(test_pred.filter(test_pred.label - test_pred.prediction != 0.0).count())


0.9989110505804938
0.8100632628051736
  
9898
9668
230
CPU times: user 162 ms, sys: 47.8 ms, total: 210 ms
Wall time: 58 s


In [ ]:
bestNaive.save('data/bestNaive')
bestLogistic.save('data/bestLogistic')
bestSVC.save('data/bestSVC')


### STEP 5.2 BAG OF WORDS  MACHINE LEARNING  (ensembles)



<B>switch to a smaller training set as the system log report OOM </B>

In [6]:
%%time

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

estimator = RandomForestClassifier()
paramMaps = ParamGridBuilder()\
            .addGrid(RandomForestClassifier.maxDepth, [5, 10, 20])\
            .addGrid(RandomForestClassifier.minInstancesPerNode, [1, 10, 50])\
            .addGrid(RandomForestClassifier.numTrees, [10, 20, 50])\
            .addGrid(RandomForestClassifier.subsamplingRate, [0.1, 0.5, 1.0])\
            .build()
evaluator = BinaryClassificationEvaluator(metricName = "areaUnderROC")
# evaluator = MulticlassClassificationEvaluator(metricName = "f1")

crossval = CrossValidator(estimator = estimator,estimatorParamMaps = paramMaps, evaluator = evaluator, numFolds = 5)
cvModel = crossval.fit(train_small)
print(cvModel.avgMetrics)


[0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378762165, 0.616136378

In [7]:
print(crossval.explainParams())

collectSubModels: Param for whether to collect a list of sub-models trained during tuning. If set to false, then only the single best sub-model will be available after fitting. If set to true, then all sub-models will be available. Warning: For large models, collecting all sub-models can cause OOMs on the Spark driver. (default: False)
estimator: estimator to be cross-validated (current: RandomForestClassifier_05d9cd4cc04e)
estimatorParamMaps: estimator param maps (current: [{Param(parent='undefined', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5, Param(parent='undefined', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1, Param(parent='undefined', name='numTrees', doc='Number of trees to train (>= 1).'): 10,

In [18]:
%%time

bestRandomForest = cvModel.bestModel
train_pred = bestRandomForest.transform(train_balanced)
train_auc = evaluator.evaluate(train_pred)
print(train_auc)
test_pred = bestRandomForest.transform(test)
test_auc = evaluator.evaluate(test_pred)
print(test_auc)
print("  ")
print(test_pred.count())
print(test_pred.filter(test_pred.label - test_pred.prediction == 0).count())
print(test_pred.filter(test_pred.label - test_pred.prediction != 0.0).count())


0.6316703792221229
0.6259974405469813
  
9898
9785
113
CPU times: user 117 ms, sys: 3.5 ms, total: 120 ms
Wall time: 57.5 s


In [9]:
bestRandomForest.save('data/bestRandomForest')

In [10]:
%%time

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

estimator = GBTClassifier()
paramMaps = ParamGridBuilder()\
            .addGrid(GBTClassifier.maxDepth, [10, 20])\
            .addGrid(GBTClassifier.minInstancesPerNode, [5, 20])\
            .addGrid(GBTClassifier.maxIter, [20, 50])\
            .addGrid(GBTClassifier.stepSize, [0.1, 0.3])\
            .addGrid(GBTClassifier.subsamplingRate, [0.1, 0.5])\
            .build()
            
evaluator = BinaryClassificationEvaluator(metricName = "areaUnderROC")
# evaluator = MulticlassClassificationEvaluator(metricName = "f1")

crossval = CrossValidator(estimator = estimator,estimatorParamMaps = paramMaps, evaluator = evaluator, numFolds = 5)
cvModel = crossval.fit(train_small)
print(cvModel.avgMetrics)


[0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779, 0.6555980514942779]
CPU times: user 46.6 s, sys: 11 s, total: 57.6 s
Wall time: 7h 3min 32s


In [11]:
print(crossval.explainParams())

collectSubModels: Param for whether to collect a list of sub-models trained during tuning. If set to false, then only the single best sub-model will be available after fitting. If set to true, then all sub-models will be available. Warning: For large models, collecting all sub-models can cause OOMs on the Spark driver. (default: False)
estimator: estimator to be cross-validated (current: GBTClassifier_656447602adb)
estimatorParamMaps: estimator param maps (current: [{Param(parent='undefined', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10, Param(parent='undefined', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 5, Param(parent='undefined', name='maxIter', doc='max number of iterations (>= 0).'): 20, Param(pa

In [12]:
%%time

bestGBT = cvModel.bestModel
train_pred = bestGBT.transform(train_balanced)
train_auc = evaluator.evaluate(train_pred)
print(train_auc)
test_pred = bestGBT.transform(test)
test_auc = evaluator.evaluate(test_pred)
print(test_auc)
print("  ")
print(test_pred.count())
print(test_pred.filter(test_pred.label - test_pred.prediction == 0).count())
print(test_pred.filter(test_pred.label - test_pred.prediction != 0.0).count())


0.715786958978393
0.6605093582827246
  
9898
9393
505
CPU times: user 93.9 ms, sys: 6.01 ms, total: 99.9 ms
Wall time: 57.8 s


In [13]:
bestGBT.save('data/bestGBT')

### STEP 6 SIMPLE NETWORK (Multilayer perceptron classifier)

<B>not working for now,  will try this again on next unit</B>

In [ ]:
# %%time

# from pyspark.ml.classification import MultilayerPerceptronClassifier
# from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# estimator = MultilayerPerceptronClassifier()
# paramMaps = ParamGridBuilder()\
#             .addGrid(MultilayerPerceptronClassifier.maxIter, [100])\
#             .addGrid(MultilayerPerceptronClassifier.layers, [[65536, 65536,32768, 2],[65536, 16384,8192, 2],[65536, 4096, 2048, 2]])\
#             .addGrid(MultilayerPerceptronClassifier.blockSize, [128])\
#             .addGrid(MultilayerPerceptronClassifier.stepSize, [0.03])\
#             .build()

# evaluator = BinaryClassificationEvaluator(metricName = "areaUnderROC")

# crossval = CrossValidator(estimator = estimator,estimatorParamMaps = paramMaps, evaluator = evaluator, numFolds = 5)
# cvModel = crossval.fit(train_balanced)
# print(cvModel.avgMetrics)


In [ ]:
# print(crossval.explainParams())

In [ ]:
# %%time

# bestMPC = cvModel.bestModel
# train_pred = bestMPC.transform(train_balanced)
# train_auc = evaluator.evaluate(train_pred)
# print(train_auc)
# test_pred = bestMPC.transform(test)
# test_auc = evaluator.evaluate(test_pred)
# print(test_auc)
# print("  ")
# print(test_pred.count())
# print(test_pred.filter(test_pred.label - test_pred.prediction == 0).count())
# print(test_pred.filter(test_pred.label - test_pred.prediction != 0.0).count())


In [ ]:
# bestMPC.save('data/bestMPC')

<B>
Load the best model from each algorithm and compare on the prediction using hold set. The auc score shows the LogisticRegression and LinearSVC has better result, As expected, the LogisticRegression is a stable algorithm which is good for buttom line modeling. The overall scores are similar to the result using sklearn  package. 
</B>

In [21]:
from pyspark.ml.classification import GBTClassificationModel, RandomForestClassificationModel, MultilayerPerceptronClassificationModel
from pyspark.ml.classification import LogisticRegressionModel, LinearSVCModel, NaiveBayesModel

bestNaive = NaiveBayesModel.load('data/bestNaive')
bestLogistic = LogisticRegressionModel.load('data/bestLogistic')
bestSVC = LinearSVCModel.load('data/bestSVC')
bestRandomForest = RandomForestClassificationModel.load('data/bestRandomForest')
bestGBT = GBTClassificationModel.load('data/bestGBT')


In [22]:
hold_pred = bestNaive.transform(hold)
hold_auc = evaluator.evaluate(hold_pred)
print(hold_auc)

print(hold_pred.count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction == 0).count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction != 0.0).count())


0.34822667859956374
12358
12237
121


In [23]:
hold_pred = bestLogistic.transform(hold)
hold_auc = evaluator.evaluate(hold_pred)
print(hold_auc)

print(hold_pred.count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction == 0).count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction != 0.0).count())

0.8398202830668989
12358
11760
598


In [24]:
hold_pred = bestSVC.transform(hold)
hold_auc = evaluator.evaluate(hold_pred)
print(hold_auc)

print(hold_pred.count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction == 0).count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction != 0.0).count())

0.8627146313753249
12358
12061
297


In [25]:
hold_pred = bestRandomForest.transform(hold)
hold_auc = evaluator.evaluate(hold_pred)
print(hold_auc)

print(hold_pred.count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction == 0).count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction != 0.0).count())

0.6079497344573124
12358
12236
122


In [26]:
hold_pred = bestGBT.transform(hold)
hold_auc = evaluator.evaluate(hold_pred)
print(hold_auc)

print(hold_pred.count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction == 0).count())
print(hold_pred.filter(hold_pred.label - hold_pred.prediction != 0.0).count())

0.6546233500715438
12358
11736
622


<B>Reference Imbalanced</B>

How pyspark handling unbalanced? basically under sample  
[PySpark tutorial – a case study using Random Forest on unbalanced dataset](https://weiminwang.blog/2016/06/09/pyspark-tutorial-building-a-random-forest-binary-classifier-on-unbalanced-dataset/)

[Class Imbalance in Credit Card Fraud Detection - Part 3 : Undersampling in Spark](http://blog.madhukaraphatak.com/class-imbalance-part-3/)

[Healthcare Dataset with Spark](https://towardsdatascience.com/healthcare-dataset-with-spark-6bf48019892b)

[Credit Card Fraud Detection Analysis on Imbalanced Data - Part 4](https://www.analyzeinsights.com/single-post/2017/10/23/Part-4-Credit-Card-Fraud-Detection-Analysis-on-Imbalanced-Data)


<B>Reference Spark NLP</B>

[A Comprehensive Guide to Understand and Implement Text Classification in Python](https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)  


[Ultimate Guide to Understand and; Implement Natural Language Processing (with codes in Python)](https://www.analyticsvidhya.com/blog/2017/01/ultimate-guide-to-understand-implement-natural-language-processing-codes-in-python/) 

[An implementation guide to Word2Vec using NumPy and Google Sheets](https://towardsdatascience.com/an-implementation-guide-to-word2vec-using-numpy-and-google-sheets-13445eebd281) 

[Approaching (Almost) Any NLP Problem on Kaggle](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle)

The below articles using RDD to apply NLTK, this is just for reference

[Natural language processing in Apache Spark using NLTK  part 1](https://towardsdatascience.com/natural-language-processing-in-apache-spark-using-nltk-part-1-2-58c68824f660)

[Natural language processing in Apache Spark using NLTK  part 2](https://towardsdatascience.com/natural-language-processing-in-apache-spark-using-nltk-part-2-2-5550b85f3340)